# Data Collection

# 1. Scraping the list Neighborhoods in Jaipur <a name="scraping"/>

**First of all, we need the list of Neighborhoods in Jaipur, Rajasthan.**  
**We will scrape the pincodes from <a href="https://www.mapsofindia.com/jaipur/localities/" target="blank_">mapsofindia.com</a>**

Import the required Libraries

In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import googlemaps
import json

In [3]:
url = 'https://www.mapsofindia.com/jaipur/localities'
page = requests.get(url)
page

<Response [200]>

In [4]:
soup = BeautifulSoup(page.text,'html.parser')
list_ = soup.find_all('div', class_ = 'intrl_links')

In [5]:
localities = []
for item in list_:
    for listItem in item.find_all('li'):
        localities.append(listItem.string)

Load the googlegeocoding API key

In [8]:
API_KEY = None
with open('/home/hotpie/Projects/keys/API_KEYS.json','r') as keyFile:
    f = json.load(keyFile)
    
API_KEY = f['google-api-key']
gmaps = googlemaps.Client(key=API_KEY)

We will define a function to fetch the latitudes and logitudes for the respective Pincodes.

In [7]:
def getLatitudeLongitude(locality):
    g = gmaps.geocode('{}, Jaipur, Rajasthan'.format(locality))
    lat_lng = [g[0]['geometry']['location']['lat'], g[0]['geometry']['location']['lng']]
    print('Location: {}, Lat: {}, Long: {}'.format(
        locality,
        lat_lng[0],
        lat_lng[1]))
    return lat_lng

Now we will call our function and store the latitude and logitude in a list.  
**This might take a while.**

In [8]:
latlong = [getLatitudeLongitude(item) for item in localities]

Location: Adarsh Nagar, Lat: 26.9018769, Long: 75.8271749
Location: Agra Road, Lat: 26.9109135, Long: 75.83637
Location: Ajmer Road, Lat: 26.8938114, Long: 75.74720839999999
Location: Ajmeri Gate, Lat: 26.9161157, Long: 75.8162613
Location: Ambabari, Lat: 26.9444814, Long: 75.7792992
Location: Amer Road, Lat: 26.9641483, Long: 75.8466641
Location: Bais Godam, Lat: 26.8971351, Long: 75.7863962
Location: Bajaj Nagar, Lat: 26.8657486, Long: 75.8013086
Location: Bani Park, Lat: 26.930559, Long: 75.793611
Location: Bapu Bazaar, Lat: 26.9163509, Long: 75.823275
Location: Bapu Nagar, Lat: 26.8898252, Long: 75.8083612
Location: Barkat Nagar, Lat: 26.8790027, Long: 75.79072699999999
Location: Bhawani Singh Road, Lat: 26.896874, Long: 75.8043331
Location: Biseswarji, Lat: 26.9194772, Long: 75.8236163
Location: Brahmapuri, Lat: 26.9372789, Long: 75.8249742
Location: Chandpol, Lat: 26.9242026, Long: 75.81344639999999
Location: Civil Lines, Lat: 26.9059311, Long: 75.78443829999999
Location: Durgapu

Let's Convert the list to a Dataframe

In [9]:
coordinates = pd.DataFrame(latlong)

Now that we have all the data we need, We'll move on to combine it into a single dataframe and export it as a csv file for further usage.

In [10]:
JaipurData = pd.DataFrame({
    'Neighborhood': localities,
    'latitude' : coordinates[0],
    'longitude': coordinates[1]
})

In [11]:
JaipurData

,Neighborhood,latitude,longitude
0,Adarsh Nagar,26.901877,75.827175
1,Agra Road,26.910913,75.836370
2,Ajmer Road,26.893811,75.747208
3,Ajmeri Gate,26.916116,75.816261
4,Ambabari,26.944481,75.779299
...,...,...,...
60,Tonk Road,26.831362,75.794138
61,Transport Nagar,26.909178,75.845694
62,Vaishali Nagar,26.904775,75.748864
63,Vidhyadhar Nagar,26.962108,75.781627


In [12]:
JaipurData.to_csv('JaipurGeoCodedLocalities.csv', index=False)

# 2. Restautrants

**Now that we have all the Neighborhoods, lets get a list of all the restaurants
For this task, we will use ZOMATO API to get a list of all the restaurents in Jaipur.**

In [3]:
df = pd.read_csv('JaipurGeoCodedLocalities.csv')

In [9]:
ZOMATO_API_KEY = f['zomato_API_KEY']

In [110]:
headers = {"User-agent": "curl/7.43.0", "Accept": "application/json","user-key": "{}".format(ZOMATO_API_KEY)}
url = 'https://developers.zomato.com/api/v2.1/search?entity_id=10&entity_type=city&start=0&count=100'
response = requests.get(url, headers=headers).json()
response['restaurants']

[{'restaurant': {'R': {'has_menu_status': {'delivery': 1, 'takeaway': -1},
    'res_id': 101837},
   'apikey': 'e3db471c0b839d355700f2c11210a78e',
   'id': '101837',
   'name': 'Agarwal Caterers',
   'url': 'https://www.zomato.com/jaipur/agarwal-caterers-shastri-nagar?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
   'location': {'address': 'E-53, Near Science Park, Shastri Nagar, Jaipur',
    'locality': 'Shastri Nagar',
    'city': 'Jaipur',
    'city_id': 10,
    'latitude': '26.9386150000',
    'longitude': '75.7970086000',
    'zipcode': '',
    'country_id': 1,
    'locality_verbose': 'Shastri Nagar, Jaipur'},
   'switch_to_order_menu': 0,
   'cuisines': 'Mithai, Bakery',
   'timings': '7 AM to 10 PM',
   'average_cost_for_two': 200,
   'price_range': 1,
   'currency': 'Rs.',
   'highlights': ['No Seating Available',
    'Delivery',
    'Cash',
    'Takeaway Available',
    'Pure Veg',
    'Desserts and Bakes'],
   'offers': [],
   'opentable_support': 0,
   'is_zom

In [111]:
response['restaurants'][0]['restaurant']['name']

'Agarwal Caterers'

List out the columns that we actually need.

In [120]:
columns = [
    'id',
    'name',
    'restaurant_latitude',
    'restaurant_longitutde',
    'cuisines',
    'average_cost_for_two',
    'price_range',
    'has_table_booking',
    'is_table_reservation_supported',
    'has_online_delivery',
    'votes',
    'aggregate_rating'
]

Let's create a function to get the details about all the restaurants in a locality.

In [124]:
def getRestInfo():
    resList = []
    start = 0
    resultSet = int(4673/20)+1
    for count in range(0,resultSet):
        url = 'https://developers.zomato.com/api/v2.1/search?entity_id=10&entity_type=city&start={}&count=20'.format(start)
        response = requests.get(url, headers=headers).json()['restaurants']
        resList.append([(
            res['restaurant']['id'],
            res['restaurant']['name'],
            res['restaurant']['location']['latitude'],
            res['restaurant']['location']['longitude'],
            res['restaurant']['cuisines'],
            res['restaurant']['average_cost_for_two'],
            res['restaurant']['price_range'],
            res['restaurant']['has_table_booking'],
            res['restaurant']['is_table_reservation_supported'],
            res['restaurant']['has_online_delivery'],
            res['restaurant']['user_rating']['votes'],
            res['restaurant']['user_rating']['aggregate_rating']
        ) for res in response])
        print("Count:",resList)
        start = start+21
    resDf = pd.DataFrame([item for res in resList for item in res])
    resDf.columns = columns
    return resDf

In [ ]:
rest = getRestInfo()

Count: [[('101837', 'Agarwal Caterers', '26.9386150000', '75.7970086000', 'Mithai, Bakery', 200, 1, 0, 0, 1, '8061', '4.6'), ('100756', 'Agarwal Caterers', '26.9053080047', '75.7439484447', 'Mithai, North Indian, South Indian, Street Food, Chinese', 500, 2, 0, 0, 1, '2358', '4.4'), ('100242', 'Shree Agarwal Caterers', '26.9541060509', '75.7780329511', 'Mithai, Fast Food', 200, 1, 0, 0, 1, '2559', '4.5'), ('100148', 'Kanha', '26.9033843229', '75.7531802729', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1844', '4.1'), ('102534', "GVD Soni's Cafe", '26.8530729316', '75.8069525287', 'Cafe, Fast Food, Beverages', 450, 1, 0, 0, 1, '1908', '4.0'), ('101736', 'Kanha', '26.8545490918', '75.8082594350', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1621', '4.1'), ('18778117', 'De Thali', '26.8732753610', '75.7759807259', 'North Indian', 550, 2, 0, 0, 1, '928', '4.1'), ('100149', 'Kanha', '26.8646163797', '75.7965271175', 'North I

Count: [[('101837', 'Agarwal Caterers', '26.9386150000', '75.7970086000', 'Mithai, Bakery', 200, 1, 0, 0, 1, '8061', '4.6'), ('100756', 'Agarwal Caterers', '26.9053080047', '75.7439484447', 'Mithai, North Indian, South Indian, Street Food, Chinese', 500, 2, 0, 0, 1, '2358', '4.4'), ('100242', 'Shree Agarwal Caterers', '26.9541060509', '75.7780329511', 'Mithai, Fast Food', 200, 1, 0, 0, 1, '2559', '4.5'), ('100148', 'Kanha', '26.9033843229', '75.7531802729', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1844', '4.1'), ('102534', "GVD Soni's Cafe", '26.8530729316', '75.8069525287', 'Cafe, Fast Food, Beverages', 450, 1, 0, 0, 1, '1908', '4.0'), ('101736', 'Kanha', '26.8545490918', '75.8082594350', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1621', '4.1'), ('18778117', 'De Thali', '26.8732753610', '75.7759807259', 'North Indian', 550, 2, 0, 0, 1, '928', '4.1'), ('100149', 'Kanha', '26.8646163797', '75.7965271175', 'North I

Count: [[('101837', 'Agarwal Caterers', '26.9386150000', '75.7970086000', 'Mithai, Bakery', 200, 1, 0, 0, 1, '8061', '4.6'), ('100756', 'Agarwal Caterers', '26.9053080047', '75.7439484447', 'Mithai, North Indian, South Indian, Street Food, Chinese', 500, 2, 0, 0, 1, '2358', '4.4'), ('100242', 'Shree Agarwal Caterers', '26.9541060509', '75.7780329511', 'Mithai, Fast Food', 200, 1, 0, 0, 1, '2559', '4.5'), ('100148', 'Kanha', '26.9033843229', '75.7531802729', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1844', '4.1'), ('102534', "GVD Soni's Cafe", '26.8530729316', '75.8069525287', 'Cafe, Fast Food, Beverages', 450, 1, 0, 0, 1, '1908', '4.0'), ('101736', 'Kanha', '26.8545490918', '75.8082594350', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1621', '4.1'), ('18778117', 'De Thali', '26.8732753610', '75.7759807259', 'North Indian', 550, 2, 0, 0, 1, '928', '4.1'), ('100149', 'Kanha', '26.8646163797', '75.7965271175', 'North I

Count: [[('101837', 'Agarwal Caterers', '26.9386150000', '75.7970086000', 'Mithai, Bakery', 200, 1, 0, 0, 1, '8061', '4.6'), ('100756', 'Agarwal Caterers', '26.9053080047', '75.7439484447', 'Mithai, North Indian, South Indian, Street Food, Chinese', 500, 2, 0, 0, 1, '2358', '4.4'), ('100242', 'Shree Agarwal Caterers', '26.9541060509', '75.7780329511', 'Mithai, Fast Food', 200, 1, 0, 0, 1, '2559', '4.5'), ('100148', 'Kanha', '26.9033843229', '75.7531802729', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1844', '4.1'), ('102534', "GVD Soni's Cafe", '26.8530729316', '75.8069525287', 'Cafe, Fast Food, Beverages', 450, 1, 0, 0, 1, '1908', '4.0'), ('101736', 'Kanha', '26.8545490918', '75.8082594350', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1621', '4.1'), ('18778117', 'De Thali', '26.8732753610', '75.7759807259', 'North Indian', 550, 2, 0, 0, 1, '928', '4.1'), ('100149', 'Kanha', '26.8646163797', '75.7965271175', 'North I

Count: [[('101837', 'Agarwal Caterers', '26.9386150000', '75.7970086000', 'Mithai, Bakery', 200, 1, 0, 0, 1, '8061', '4.6'), ('100756', 'Agarwal Caterers', '26.9053080047', '75.7439484447', 'Mithai, North Indian, South Indian, Street Food, Chinese', 500, 2, 0, 0, 1, '2358', '4.4'), ('100242', 'Shree Agarwal Caterers', '26.9541060509', '75.7780329511', 'Mithai, Fast Food', 200, 1, 0, 0, 1, '2559', '4.5'), ('100148', 'Kanha', '26.9033843229', '75.7531802729', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1844', '4.1'), ('102534', "GVD Soni's Cafe", '26.8530729316', '75.8069525287', 'Cafe, Fast Food, Beverages', 450, 1, 0, 0, 1, '1908', '4.0'), ('101736', 'Kanha', '26.8545490918', '75.8082594350', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1621', '4.1'), ('18778117', 'De Thali', '26.8732753610', '75.7759807259', 'North Indian', 550, 2, 0, 0, 1, '928', '4.1'), ('100149', 'Kanha', '26.8646163797', '75.7965271175', 'North I

...

Count: [[('101837', 'Agarwal Caterers', '26.9386150000', '75.7970086000', 'Mithai, Bakery', 200, 1, 0, 0, 1, '8061', '4.6'), ('100756', 'Agarwal Caterers', '26.9053080047', '75.7439484447', 'Mithai, North Indian, South Indian, Street Food, Chinese', 500, 2, 0, 0, 1, '2358', '4.4'), ('100242', 'Shree Agarwal Caterers', '26.9541060509', '75.7780329511', 'Mithai, Fast Food', 200, 1, 0, 0, 1, '2559', '4.5'), ('100148', 'Kanha', '26.9033843229', '75.7531802729', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1844', '4.1'), ('102534', "GVD Soni's Cafe", '26.8530729316', '75.8069525287', 'Cafe, Fast Food, Beverages', 450, 1, 0, 0, 1, '1908', '4.0'), ('101736', 'Kanha', '26.8545490918', '75.8082594350', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1621', '4.1'), ('18778117', 'De Thali', '26.8732753610', '75.7759807259', 'North Indian', 550, 2, 0, 0, 1, '928', '4.1'), ('100149', 'Kanha', '26.8646163797', '75.7965271175', 'North I

Count: [[('101837', 'Agarwal Caterers', '26.9386150000', '75.7970086000', 'Mithai, Bakery', 200, 1, 0, 0, 1, '8061', '4.6'), ('100756', 'Agarwal Caterers', '26.9053080047', '75.7439484447', 'Mithai, North Indian, South Indian, Street Food, Chinese', 500, 2, 0, 0, 1, '2358', '4.4'), ('100242', 'Shree Agarwal Caterers', '26.9541060509', '75.7780329511', 'Mithai, Fast Food', 200, 1, 0, 0, 1, '2559', '4.5'), ('100148', 'Kanha', '26.9033843229', '75.7531802729', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1844', '4.1'), ('102534', "GVD Soni's Cafe", '26.8530729316', '75.8069525287', 'Cafe, Fast Food, Beverages', 450, 1, 0, 0, 1, '1908', '4.0'), ('101736', 'Kanha', '26.8545490918', '75.8082594350', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1621', '4.1'), ('18778117', 'De Thali', '26.8732753610', '75.7759807259', 'North Indian', 550, 2, 0, 0, 1, '928', '4.1'), ('100149', 'Kanha', '26.8646163797', '75.7965271175', 'North I

Count: [[('101837', 'Agarwal Caterers', '26.9386150000', '75.7970086000', 'Mithai, Bakery', 200, 1, 0, 0, 1, '8061', '4.6'), ('100756', 'Agarwal Caterers', '26.9053080047', '75.7439484447', 'Mithai, North Indian, South Indian, Street Food, Chinese', 500, 2, 0, 0, 1, '2358', '4.4'), ('100242', 'Shree Agarwal Caterers', '26.9541060509', '75.7780329511', 'Mithai, Fast Food', 200, 1, 0, 0, 1, '2559', '4.5'), ('100148', 'Kanha', '26.9033843229', '75.7531802729', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1844', '4.1'), ('102534', "GVD Soni's Cafe", '26.8530729316', '75.8069525287', 'Cafe, Fast Food, Beverages', 450, 1, 0, 0, 1, '1908', '4.0'), ('101736', 'Kanha', '26.8545490918', '75.8082594350', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1621', '4.1'), ('18778117', 'De Thali', '26.8732753610', '75.7759807259', 'North Indian', 550, 2, 0, 0, 1, '928', '4.1'), ('100149', 'Kanha', '26.8646163797', '75.7965271175', 'North I

Count: [[('101837', 'Agarwal Caterers', '26.9386150000', '75.7970086000', 'Mithai, Bakery', 200, 1, 0, 0, 1, '8061', '4.6'), ('100756', 'Agarwal Caterers', '26.9053080047', '75.7439484447', 'Mithai, North Indian, South Indian, Street Food, Chinese', 500, 2, 0, 0, 1, '2358', '4.4'), ('100242', 'Shree Agarwal Caterers', '26.9541060509', '75.7780329511', 'Mithai, Fast Food', 200, 1, 0, 0, 1, '2559', '4.5'), ('100148', 'Kanha', '26.9033843229', '75.7531802729', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1844', '4.1'), ('102534', "GVD Soni's Cafe", '26.8530729316', '75.8069525287', 'Cafe, Fast Food, Beverages', 450, 1, 0, 0, 1, '1908', '4.0'), ('101736', 'Kanha', '26.8545490918', '75.8082594350', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1621', '4.1'), ('18778117', 'De Thali', '26.8732753610', '75.7759807259', 'North Indian', 550, 2, 0, 0, 1, '928', '4.1'), ('100149', 'Kanha', '26.8646163797', '75.7965271175', 'North I

Count: [[('101837', 'Agarwal Caterers', '26.9386150000', '75.7970086000', 'Mithai, Bakery', 200, 1, 0, 0, 1, '8061', '4.6'), ('100756', 'Agarwal Caterers', '26.9053080047', '75.7439484447', 'Mithai, North Indian, South Indian, Street Food, Chinese', 500, 2, 0, 0, 1, '2358', '4.4'), ('100242', 'Shree Agarwal Caterers', '26.9541060509', '75.7780329511', 'Mithai, Fast Food', 200, 1, 0, 0, 1, '2559', '4.5'), ('100148', 'Kanha', '26.9033843229', '75.7531802729', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1844', '4.1'), ('102534', "GVD Soni's Cafe", '26.8530729316', '75.8069525287', 'Cafe, Fast Food, Beverages', 450, 1, 0, 0, 1, '1908', '4.0'), ('101736', 'Kanha', '26.8545490918', '75.8082594350', 'North Indian, South Indian, Fast Food, Chinese, Beverages', 500, 2, 0, 0, 1, '1621', '4.1'), ('18778117', 'De Thali', '26.8732753610', '75.7759807259', 'North Indian', 550, 2, 0, 0, 1, '928', '4.1'), ('100149', 'Kanha', '26.8646163797', '75.7965271175', 'North I

In [123]:
rest

,id,name,restaurant_latitude,restaurant_longitutde,cuisines,average_cost_for_two,price_range,has_table_booking,is_table_reservation_supported,has_online_delivery,votes,aggregate_rating
0,101837,Agarwal Caterers,26.9386150000,75.7970086000,"Mithai, Bakery",200,1,0,0,1,8061,4.6
1,100756,Agarwal Caterers,26.9053080047,75.7439484447,"Mithai, North Indian, South Indian, Street Foo...",500,2,0,0,1,2358,4.4
2,100242,Shree Agarwal Caterers,26.9541060509,75.7780329511,"Mithai, Fast Food",200,1,0,0,1,2559,4.5
3,100148,Kanha,26.9033843229,75.7531802729,"North Indian, South Indian, Fast Food, Chinese...",500,2,0,0,1,1844,4.1
4,102534,GVD Soni's Cafe,26.8530729316,75.8069525287,"Cafe, Fast Food, Beverages",450,1,0,0,1,1908,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
75,18475655,Masala Trails,26.8686328154,75.7835033163,"North Indian, Italian",1200,3,0,0,1,907,4.0
76,18619564,La Palma Cafe and Lounge Bar,26.9140260523,75.8047323301,"Italian, European, Continental",1400,3,1,1,1,825,4.2
77,100125,Hanuman Dhaba,26.9535702026,75.7715966553,North Indian,350,1,0,0,1,932,3.9
78,102620,Curious Life Coffee Roasters- Sarojini Marg,26.9092856227,75.8086004108,Cafe,600,2,0,0,0,879,4.4
